<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Bubble_Dew_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
from dotmap import DotMap
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32 bit - enable 64 bit - double precision

from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

from tools.tree_array_transform2 import VSC, Comp, Range, RangeArray
import tools.che as che
R=8.314 # J/(mol K)

In [3]:
p = che.Props(['Methanol', 'Ethanol'])

In [4]:
def bubble_T(c):
    return jnp.sum(c.x * p.Pvap(c.T))-c.P

In [5]:
c=DotMap()
c.x = jnp.array([0.5,0.5])
c.T = Range(300., 273., 400.)
c.P = 101325.
c.T = Range(300., 273.15, 400.)
vsc = VSC(c, bubble_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

343.76648202145736


In [7]:
def bubble_T_Vec(c):
    r=DotMap()
    r.me = 34.
    return jnp.sum(c.x*p.PvapVec(c.T),1) - c.P, r

In [8]:
x1 = jnp.linspace(0,1,20)
x2 = 1-x1

c = DotMap()
c.x = jnp.stack([x1,x2],1)
c.P = 101325
c.T = RangeArray(300*jnp.ones_like(x1), 273.15, 400.)
vsc = VSC(c, bubble_T_Vec)
vsc.solve(verbosity=0)
print(vsc.v.T)

[351.45840044 350.55776211 349.68058589 348.82599244 347.99313777
 347.18121234 346.38943999 345.6170769  344.86341046 344.12775818
 343.40946653 342.70790984 342.02248916 341.35263117 340.69778711
 340.0574317  339.43106212 338.81819705 338.21837565 337.63115666]


In [9]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=x1, y=vsc.v.T, name='Liq')
# fig.add_scatter(x=y1, y=Ptot, name='Vap')
fig.update_xaxes(showline=True, linewidth=1, title_text='x1, y1')
fig.update_yaxes(showline=True, linewidth=1, title_text='Temperature (K)')
fig.update_layout(width=800,height=600, title_text=f'MeOH / EtOH ({c.P} P)', title_x=0.5)